<a href="https://colab.research.google.com/github/rcnewman/cv_gesture_recognition_posenet/blob/master/PoseNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# PoseNet-python Notebook Source: https://colab.research.google.com/drive/1Ha7-lt-WwcCx961cuatjOzQ-7GLY91aQ#scrollTo=k_qlzhSXN3D7
# install
!git clone https://github.com/rcnewman/posenet-python.git #fork from https://www.github.com/rwightman/posenet-python 
%cd posenet-python
!pip3 install tensorflow-gpu==1.15
!pip3 install scipy
!pip3 install pyyaml
!pip3 install opencv-python==3.4.5.20
!pip3 install ipykernel

Cloning into 'posenet-python'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 123 (delta 0), reused 1 (delta 0), pack-reused 119
Receiving objects: 100% (123/123), 38.00 KiB | 1.81 MiB/s, done.
Resolving deltas: 100% (68/68), done.
/content/posenet-python/posenet-python


In [4]:
# PoseNet python sample program
import tensorflow as tf
import cv2
import time
import argparse
import os
import posenet

print('INIT:')
from google.colab import drive
drive.mount('/content/drive')
drivepath="/content/drive/My Drive/POSENET/"
driveinfile = drivepath + 'input.mp4'
driveoutfile = drivepath + 'output.mp4'

# VideoReaderWriter
cap = cv2.VideoCapture(driveinfile)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc('h','2','6', '4')
video = cv2.VideoWriter(driveoutfile, fourcc, fps, (width, height))

model = 101
###scale_factor = 1.0
scale_factor = 0.4

with tf.Session() as sess:
    print('MODEL-INIT:')
    ####model_cfg, model_outputs = posenet.load_model(args.model, sess)
    model_cfg, model_outputs = posenet.load_model(model, sess)
    output_stride = model_cfg['output_stride']
    start = time.time()
    print('START:')

    incnt = 0
    while True:
        incnt = incnt + 1
        try: input_image, draw_image, output_scale = posenet.read_cap(
                cap, scale_factor=scale_factor, output_stride=output_stride)
        except:break
        heatmaps_result, offsets_result, displacement_fwd_result, displacement_bwd_result = sess.run(
            model_outputs,
            feed_dict={'image:0': input_image}
        )
        pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multiple_poses(
            heatmaps_result.squeeze(axis=0),
            offsets_result.squeeze(axis=0),
            displacement_fwd_result.squeeze(axis=0),
            displacement_bwd_result.squeeze(axis=0),
            output_stride=output_stride,
            max_pose_detections=10,
            min_pose_score=0.25)

        keypoint_coords *= output_scale
        
        # print(keypoint_coords)
        
        draw_image = posenet.draw_skel_and_kp(
                draw_image, pose_scores, keypoint_scores, keypoint_coords,
                min_pose_score=0.25, min_part_score=0.25)

        video.write(draw_image)
        if incnt % 100 == 0:        
            print("cnt=", incnt, "fps=", incnt / (time.time() - start) )

        if False:
            #debug
            for pi in range(len(pose_scores)):
                if pose_scores[pi] == 0.:
                    break
                print('Pose #%d, score = %f' % (pi, pose_scores[pi]))
                for ki, (s, c) in enumerate(zip(keypoint_scores[pi, :], keypoint_coords[pi, :, :])):
                    print('Keypoint %s, score = %f, coord = %s' % (posenet.PART_NAMES[ki], s, c))
video.release()
cap.release()
cv2.destroyAllWindows()
print('END:')


INIT:
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
MODEL-INIT:
Cannot find model file ./_models/model-mobilenet_v1_101.pb, converting from tfjs...





Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./_models/checkpoints/model-mobilenet_v1_101.ckpt
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 62 variables.
INFO:tensorflow:Converted 62 variables to const ops.
START:
cnt= 100 fps= 5.481804088588368
cnt= 200 fps= 5.467315737130568
END:
